# Topic Modeling

## Introduzione

Con il Topic Modeling è possibile individuare, in modo non supervisionato, i principali argomenti trattati all'interno di un corpus di documenti. Ogni **topic** è descritto come una serie di token e/o frasi rilevanti che possono essere utilizzate, successivamente, per indicizzare il documento. 

##### Alcuni impieghi

- motori di ricerca cognitivi: si taggano i contenuti con le parole chiave dei topic principali, per sfruttarle poi in fase di ricerca
- customer service: si riesce ad individuare i topic principali discussi dai clienti e capirne più facilmente e rapidamente i bisogni
- social network: per individuare gli argomenti principali trattati dagli utenti
- sentiment analysis: per individuare concetti che sono espressione del sentiment del contenuto

##### Le 2 tecniche principali sono:
 - Latent Semantic Analysis (LSA)
 - Latent Dirichlet Allocation (LDA)

## Import Libraries

In [1]:
import gensim

from gensim.utils import simple_preprocess
from gensim import corpora, models

In [2]:
import re
import string

from pprint import pprint

La libreria **spacy** sarà utilizzata solo per estrarre il lemma dalle parole operando così una diminuzione dello spazio dimensionale.

In [3]:
import spacy

In [4]:
nlp = spacy.load('it_core_news_sm')

In [5]:
with open('stopwords.txt', 'r') as f:
    stopwords = f.read()
    
stopwords = stopwords.split("\n")

# print(type(stopwords))

## Import dataset

In [6]:
with open('dataset_little.txt', 'r') as f:
    documents = f.readlines()

In [7]:
print(f"""Il dataset si compone di n.{len(documents)} documenti.""")

Il dataset si compone di n.72 documenti.


In [8]:
iter_documents = iter(documents)

In [9]:
test_document = next(iter_documents)[64:97]
test_document

'è una società calcistica italiana'

## Preprocessing

Il preprocessing si compone di alcuni semplici step:
1. rimozione punteggiatura, numeri, ... , in sostanza restano solo le parole
2. riduzione in minuscolo (operazione che può interferire con il NER di Spacy
3. vengono rimosse le stopwords
4. costruire bigrammi e trigrammi per individuare i pattern di parole più frequenti
5. estrazione del lemma delle parole

In [10]:
def estrai_solo_parole(document:str) -> str:
    
    document = re.sub("\W|\d", " ", document)
    document = re.sub("[ ]+", " ", document)
    document = document.lower()
    
    return document

In [11]:
def remove_stopwords(document:str, stopwords:list) -> str:
    
    list_document = document.split(" ")
    
    list_document = [word for word in list_document if word not in stopwords]
    
    document = " ".join(list_document)
    
    return document

In [12]:
def get_lemma(document:str) -> str:
    
    """Versione che restituisce il lemma solo per i verbi"""
    
    doc = nlp(document)
    
    tokens = [token.lemma_ if token.pos_ == 'VERB' else token.text for token in doc]
    
    document = " ".join(tokens)
    
    return document

#### Raggruppamento nella funzione di preprocessing

In [13]:
def preprocessing(document:str, stopwords:list) -> str:
    
    """Questa funzione raggruppa tutte le precedenti"""
    
    document = estrai_solo_parole(document)
    document = remove_stopwords(document, stopwords)
#     document = get_lemma(document)
    
    return document

### Costruzione del _corpus_ dei documenti

In [14]:
corpus = [preprocessing(document, stopwords).split() for document in iter_documents]

In [15]:
print(corpus[0][:10])

['tecnologia', 'star', 'trek', 'universo', 'star', 'trek', 'serie', 'televisiva', 'fantascientifica', 'originariamente']


### Estrazione dei pattern di parole più frequenti

In [16]:
min_count=5

"""min_count : float, optional
    Ignore all words and bigrams with total collected count lower than this value.
"""


threshold=100

"""threshold : float, optional
    Represent a score threshold for forming the phrases (higher means fewer phrases).
"""

bigram = gensim.models.Phrases(corpus, min_count=min_count, threshold=threshold) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus], threshold=threshold)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:
def get_words_pattern(corpus:list, 
                      bigram_mod:gensim.models.phrases.FrozenPhrases, 
                      trigram_mod:gensim.models.phrases.FrozenPhrases) -> list:
    
    """Va applicato a tutto il corpus documenti.
Il corpus deve essere una lista annidata di tokens, 
quindi una lista di token per ogni lista di documenti"""
    
    pattern = [trigram_mod[bigram_mod[doc]] for doc in corpus]
    
    return pattern

In [18]:
pattern = get_words_pattern(corpus, bigram_mod, trigram_mod)

In [19]:
print(pattern[0][:10])

['tecnologia', 'star_trek', 'universo', 'star_trek', 'serie', 'televisiva', 'fantascientifica', 'originariamente', 'ideata', 'gene']


In [20]:
# [[(token.text, token.lemma_ ) for token in nlp(" ".join(doc))] for doc in pattern]

In [21]:
# # Build the bigram and trigram models
# print("\n bigram")
# bigram = gensim.models.Phrases(corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
# pprint(bigram.export_phrases())

# print("\n trigram")
# trigram = gensim.models.Phrases(bigram[corpus], threshold=100)
# pprint(trigram.export_phrases())

# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# pattern = [trigram_mod[bigram_mod[doc]] for doc in corpus]

# print("\n patterns")
# print(pattern[0][:10])

### Dizionario delle parole (Bag-of-Words)

Ottenute le parole più frequenti si può costruire il vocabolario di parole (**Bag-of-Words**) per l'addestramento dell'algoritmo

In [22]:
id2word = corpora.Dictionary(pattern)

In [23]:
print(id2word)

Dictionary(20472 unique tokens: ['accadere', 'accurata', 'addirittura', 'alimentari', 'alphahypertreklois']...)


## Costruzione del corpus per l'addestramento dell'algoritmo

In [24]:
corpus_doc2bow = [id2word.doc2bow(trigram_mod[bigram_mod[doc]]) for doc in pattern]

In [25]:
print(id2word[(corpus_doc2bow[0][0][0])])
print((corpus_doc2bow[0][0]))

accadere
(0, 1)


In [26]:
tfidf_model = gensim.models.TfidfModel(corpus_doc2bow, id2word=id2word)

In [27]:
print(id2word[(corpus_doc2bow[0][0][0])])
print(tfidf_model[corpus_doc2bow][0][0])

accadere
(0, 0.03476187341468142)


### Addestramento dell'algoritmo (Lda)

In [28]:
num_topics = 3

lda_model = gensim.models.LdaModel(tfidf_model[corpus_doc2bow], \
                                   num_topics=num_topics, \
                                   id2word=id2word, \
                                   passes=4, \
                                   random_state=1)

In [29]:
# lda_model.show_topics(num_words=5)

In [30]:
"""
Prime 5 parole che identificano ogni Topic
"""
for i in range(len(lda_model.show_topics())):
    print(f"\nTopic n.{i}")
    print([k for k,v in lda_model.show_topic(i, topn=5)])


Topic n.0
['salute', 'wimax', 'sci', 'gpp', 'sport']

Topic n.1
['podcast', 'amd', 'linguaggio', 'zeno', 'videotelefonia']

Topic n.2
['pugno', 'saluto', 'gmail', 'chopin', 'sport_motoristici']


## Valutazione del modello

In [31]:
from gensim.models import CoherenceModel

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(tfidf_model[corpus_doc2bow]))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, \
                                     texts=[trigram_mod[bigram_mod[doc]] for doc in pattern], \
                                     dictionary=id2word, \
                                     coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.79921384325789

Coherence Score:  0.6541796373629191


## Prediction su nuovo documento

In [33]:
new_document = test_document
print(new_document)

è una società calcistica italiana


In [34]:
corpus = preprocessing(new_document, stopwords).split()

In [35]:
corpus = [id2word.doc2bow(trigram_mod[bigram_mod[corpus]])]

In [36]:
corpus = tfidf_model[corpus]

In [37]:
topics = lda_model[corpus][0]

In [38]:
topics

[(0, 0.6441914), (1, 0.19824927), (2, 0.15755938)]

In [39]:
topics = lda_model.get_document_topics(corpus)[0]

In [40]:
topics

[(0, 0.644034), (1, 0.19840541), (2, 0.15756062)]

#### Estrazione Topic

In [41]:
def get_top_n_topics(topics:list, probability:float=0.0, n:int=1):
    
    if n <= 0:
        n = 1
    
    if probability > 0:
        topic_list = [k for k,v in sorted(topics , key=lambda item: item[1], reverse=True) if v >= probability]
    else:
        topic_list = [k for k,v in sorted(topics , key=lambda item: item[1], reverse=True)]
        topic_list = topic_list[:n]
    
    return topic_list

In [42]:
get_top_n_topics(topics, n=1)

[0]

## Salvataggio modello

In [43]:
# store all trained models to disk
lda_model.save('lda.model')
tfidf_model.save('tfidf.model')
id2word.save('id2word.dictionary')

In [44]:
lda_model = gensim.models.LdaModel.load('lda.model')